In [400]:
import csv
import sys
import pandas as pd
import numpy as np
import copy
from operator import itemgetter
dataPath = "./INTEGRATED-DATASET.csv"



dataSet = pd.read_csv(dataPath)
dataSet.columns = ["Name", "Borough", "Type", "InspectionSeason", "Reason", "Critical", "Score", "Grade", "GradeSeason"]
dataSet
min_supp = 0*len(dataSet)
min_conf = 0.1*len(dataSet)



In [401]:
def get_Lk(one_item_set):
    Union_Lk = []
    Union_Lk.append(one_item_set)
    k=1
    while len(Union_Lk[-1])!=0 and k<=4:
        Ck = apriori_gen(Union_Lk[-1], k)
        for idx, c in enumerate(Ck):
            cols = []
            vals = []
            for item in c:
                cols.append(item[0])
                vals.append(item[1])
            gb = dataSet.groupby(cols)
            try:
                if len(gb.get_group(tuple(vals))) < min_supp:
                    del Ck[idx]            
            except KeyError:
                del Ck[idx]            

        Union_Lk.append(Ck)
        k=k+1
    return Union_Lk

        
def apriori_gen(Lk_minus1, k):
    Ck = []
    print(len(Lk_minus1))
    for i in xrange(0, len(Lk_minus1)):
        for j in xrange(i+1, len(Lk_minus1)):
            if check_match(Lk_minus1[i], Lk_minus1[j]):
                
                new_item_set = Lk_minus1[i][:]
                new_item_set.append(Lk_minus1[j][-1])
                if k==1 or check_subset(Lk_minus1, new_item_set):
                    Ck.append(new_item_set)
#     sort(Ck) not sure if we need to sort it                
    return Ck                
def check_subset(Lk_minus1, new_item_set):
    for i in xrange(len(new_item_set)):
        subset = copy.deepcopy(new_item_set)
        del subset[i]
        if subset not in Lk_minus1:
            
            return False
    return True
            
            
        
def check_match(set1, set2):
    for i in xrange(len(set1)-1):
        
        if set1[i]!=set2[i]: return False
    if set1[-1][0]==set2[-1][0]: 
        return False
    if set1[-1][0]!=set2[-1][0] and set1[-1][1]!=set2[-1][1]: return True 

In [402]:
def get_L1(dataSet, one_item):
    for item in one_item:
        cntType = dataSet.groupby(item).count()
        one_item[item] = [x for x in one_item[item] if cntType.at[x, 'Name'] >= min_supp]

def init_one_item_set(one_item, one_item_set):
    for key in one_item:
        for val in one_item[key]:
            new_item_set = []
            new_item_set.append((key, val))
            one_item_set.append(new_item_set)

            

    
boroughs = dataSet["Borough"].unique()
grades = dataSet["Grade"].unique()
types = dataSet["Type"].unique()
gradeSeason = dataSet["GradeSeason"].unique()

gradeSeason = np.delete(gradeSeason, gradeSeason.shape[0]-1).tolist()
types = np.delete(types, types.shape[0]-1).tolist()
grades = np.delete(grades, grades.shape[0]-1).tolist()
boroughs = np.delete(boroughs, boroughs.shape[0]-1).tolist()

one_item={}
one_item["Type"] = types
one_item["Borough"] = boroughs
one_item["Grade"] = grades
one_item["GradeSeason"] = gradeSeason
get_L1(dataSet, one_item)

one_item_set = []
init_one_item_set(one_item, one_item_set)

one_item_set = sorted(one_item_set, key=lambda x: (x[0],x[0][1]))
print(one_item_set)


[[('Borough', 'BRONX')], [('Borough', 'BROOKLYN')], [('Borough', 'MANHATTAN')], [('Borough', 'QUEENS')], [('Borough', 'STATEN ISLAND')], [('Grade', 'A')], [('Grade', 'B')], [('Grade', 'C')], [('Grade', 'P')], [('Grade', 'Z')], [('GradeSeason', 'Spring-GRADE DATE')], [('GradeSeason', 'Summer-GRADE DATE')], [('GradeSeason', 'Winter-GRADE DATE')], [('Type', 'African')], [('Type', 'Asian')], [('Type', 'European')], [('Type', 'NorthAmerican')], [('Type', 'Other')], [('Type', 'SouthAmerican')]]


In [403]:
Union_Lk = get_Lk(one_item_set)
# Union_Lk

19
133
388
357


In [398]:
tri = dataSet.groupby(['Borough', 'Grade', 'Type'])
tri
# print tri.get_group(('BRONX', 'B', 'SouthAmerican'))
# tri[["Type", "Borough"]].get_group(("Type", "Borough"))


In [1]:
def pick_up_cal(df, result):
    temp = df
    for index in range(len(result)):
        temp = temp[temp[result[index][0]] == result[index][1]]
    return temp.shape[0]

def pick_up_num(df, num, selected, remains, index):
    result = []
    if num > len(remains) - index:
        return []
    if num == 0:
        A = pick_up_cal(df, selected+remains)
        B = pick_up_cal(df, selected)
        return [json.dumps({"RHS": selected+remains, "LHS":selected, "Confidence": A*1.0/B})]
    result += pick_up_num(df, num, selected, remains, index+1)
    result += pick_up_num(df, num-1, selected + [remains[index]], remains[:index] + remains[index+1:], index)
    return result
    
# sample = [('Borough', 'BRONX'), ('Grade', 'A'), ('Type', 'African')]
# pick_up_cal(dataSet, sample)

# print pick_up_num(dataSet, 2, [], sample, 0)

def generate_fit(df, result, threshold):
    submit = []
    for level, value in enumerate(result):
        # under 1st Level
        if level == 0: continue
        for index, item in enumerate(value):
            # e.g item = [('Borough', 'BRONX'), ('Grades', 'A')]
            num = level + 1
            for i in range(1, num):
                submit += pick_up_num(df, i, [], item, 0)
    return submit

final = generate_fit(dataSet, Union_Lk, 0.5)

confidence = 0.6

for item in final:
    temp = json.loads(item)
    if temp["Confidence"] > confidence: print temp

NameError: name 'dataSet' is not defined